In [1]:
from pprint import pprint
from io import StringIO

# XPath

- Dokumenty XML są drzewami węzłów.
- Element od którego rozpoczyna się drzewo nazywamy korzeniem (root element)
- Wartości atomowe to takie, które nie mają ani rodziców, ani dzieci.

### Związki między węzłami

- **dzieci** _(ang. children)_ - każdy węzeł ma 0, 1 lub wiele dzieci.
- **rodzeństwo** _(ang. siblings)_ - węzły posiadające tego samego rodzica.
- **przodkowie** _(ang. ancestors)_ - węzły będące rodzicami, rodzicami rodziców itd.
- **zstępni** _(ang. descendants)_ - węzły będące dziećmi, dziećmi dzieci itd.

### Składnia wyboru elementów

```
nazwawezla - wybiera wszystkie wezly o nazwie "nazwawezla"
/          - wybiera korzeń drzewa elementów
//         - wybiera węzeł począwszy od bieżącego, niezależnie od jego pozycji
.          - wybiera bieżący węzeł
..         - wybiera rodzica bieżącego węzła
@          - wybiera atrybut węzła
```

In [2]:
xml = """
<ArrayOfExchangeRatesTable xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
    <ExchangeRatesTable lang="pl" type="mid">
        <Table>A</Table>
        <No>023/A/NBP/2019</No>
        <EffectiveDate>2019-02-01</EffectiveDate>
        <Rates>
            <Rate>
                <Currency>bat (Tajlandia)</Currency>
                <Code>THB</Code>
                <Mid>0.1190</Mid>
            </Rate>
            <Rate>
                <Currency>dolar amerykański</Currency>
                <Code>USD</Code>
                <Mid>3.7243</Mid>
            </Rate>
            <Rate>
                <Currency>dolar australijski</Currency>
                <Code>AUD</Code>
                <Mid>2.7027</Mid>
            </Rate>
        </Rates>
    </ExchangeRatesTable>
</ArrayOfExchangeRatesTable>
"""


In [3]:
!pip install lxml
from lxml import etree

In [4]:
xml_file = StringIO(xml)
tree = etree.parse(xml_file)


### Wyszukiwanie elementów

In [5]:
r = tree.xpath('//Rate')

print(len(r))

print(r[0].xpath('Currency')[0].text)

for element in r:
    print(element.tag)

3
bat (Tajlandia)
Rate
Rate
Rate


In [ ]:
r = tree.xpath('//Rate/*')

for element in r:
    print(element.tag, element.text)

In [ ]:
r = tree.xpath('//Rates/Rate/Currency')
for element in r:
    print(element.text)

In [ ]:
r = tree.xpath('//Rates/Rate[1]/Currency')
for element in r:
    print(element.text)

In [ ]:
r = tree.xpath('//Rates/Rate[last()]/Currency')
for element in r:
    print(element.text)

In [ ]:
r = tree.xpath('//Rates/Rate[last()-1]/Currency')
for element in r:
    print(element.text)

In [ ]:
r = tree.xpath('//Rates/Rate[position()<3]/Currency')
for element in r:
    print(element.text)

In [ ]:
r = tree.xpath('/ArrayOfExchangeRatesTable/ExchangeRatesTable/Rates/Rate/Currency')
for element in r:
    print(element.text)

### Atrybuty

In [ ]:
r = tree.xpath('//ExchangeRatesTable[@lang]')
for element in r:
    print(element.tag)

In [ ]:
r = tree.xpath('//ExchangeRatesTable/@lang')
for element in r:
    print(element)
    print(type(element))

In [ ]:
r = tree.xpath('//ExchangeRatesTable/@*')
for element in r:
    print(element)

### Operatory

In [ ]:
r = tree.xpath('//Rate[Mid<3.7]/Currency')
for element in r:
    print(element.text)

# Zadanie

Plik `waluty.xml` zawiera dane n.t. średnich kursów walut NBP z przedziału dat od `2019-01-01` do `2019-03-31`.
Przetwórz plik XML do `DataFrame` zawierającego kolumny :

    - Data
    - Nazwa waluty
    - Symbol Waluty
    - Średni kurs
    
## Wariant 1

Wykonaj przetworzenie dla walut: dolar amerykański, euro, forint węgierski

## Wariant 2

Rozszerz wariant 1 tak, aby działał dla wszystkich walut z załączonego pliku


In [23]:
tree = etree.parse('data/waluty.xml')
import pandas as pd

In [30]:
r = tree.xpath('//ExchangeRatesTable/EffectiveDate')

myList = []

df = pd.DataFrame(columns=['Data','Nazwa waluty','Symbol Waluty','Sredni kurs'])

for element in r:
    date = element.text
    rates = element.xpath('../Rates/Rate')
    for rate in rates:
        currency_name = rate.xpath('./Currency')[0].text
        currency_symbol = rate.xpath('./Code')[0].text
        currency_mid = rate.xpath('./Mid')[0].text
        #print(f'Data: {date}, nazwa: {currency_name}, symbol: {currency_symbol}, sredni kurs: {currency_mid}')
        myList.append((date, currency_name,currency_symbol,currency_mid))
        
        
df = pd.DataFrame(myList, columns=['Data','Nazwa waluty','Symbol Waluty','Sredni kurs'])
df
        

,Data,Nazwa waluty,Symbol Waluty,Sredni kurs
0,2019-01-02,bat (Tajlandia),THB,0.1166
1,2019-01-02,dolar amerykański,USD,3.7619
2,2019-01-02,dolar australijski,AUD,2.6393
3,2019-01-02,dolar Hongkongu,HKD,0.4801
4,2019-01-02,dolar kanadyjski,CAD,2.7608
5,2019-01-02,dolar nowozelandzki,NZD,2.5246
6,2019-01-02,dolar singapurski,SGD,2.7601
7,2019-01-02,euro,EUR,4.3016
8,2019-01-02,forint (Węgry),HUF,0.013353
9,2019-01-02,frank szwajcarski,CHF,3.8242
